In [27]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta 
from datetime import date
from datetime import datetime

from dateutil.relativedelta import relativedelta

## Global variable, set company

In [28]:
company = "Fidelity"  #<- change variable
portfolios_number=500


## Lists of funds

In [2]:

Allianz_fund_list = ['Allianz Asian Fund', 'Allianz Balanced Fund', 'Allianz Capital Stable Fund', 'Allianz Flexi Balanced Fund', 'Allianz Greater China Fund', 'Allianz Growth Fund', 'Allianz Hong Kong Fund', 'Allianz MPF Age 65 Plus Fund', 'Allianz MPF Conservative Fund', 'Allianz MPF Core Accumulation Fund', 'Allianz Oriental Pacific Fund', 'Allianz RMB Money Market Fund', 'Allianz Stable Growth Fund']
Investco_fund_list = ['Investco Age 65 Plus Fund', 'Investco Asian Equity Fund', 'Investco Balanced Fund', 'Investco Capital Stable Fund', 'Investco Core Accumulation Fund', 'Investco Global Bond Fund', 'Investco Growth Fund', 'Investco Hong Kong and China Equity Fund', 'Investco Hang Seng Index Tracking Fund', 'Investco MPF Conservative Fund', 'Investco RMB Bond Fund']
Fidelity_fund_list=['Fidelity Age 65 Plus Fund', 'Fidelity Asia Pacific Equity Fund', 'Fidelity Balanced Fund', 'Fidelity Capital Stable Fund', 'Fidelity Core Accumulation Fund', 'Fidelity Global Equity Fund', 'Fidelity Growth Fund', 'Fidelity Hong Kong Bond Fund', 'Fidelity Hong Kong Equity Fund', 'Fidelity Hong Kong Tracker Fund', 'Fidelity MPF Conservative Fund', 'Fidelity RMB Bond Fund', 'Fidelity World Bond Fund']

Allianz_chinese_fund_list = ['Allianz Asian Fund', 'Allianz Greater China Fund', 'Allianz Oriental Pacific Fund', 'Allianz RMB Money Market Fund']
Allianz_us_fund_list = ['Allianz Capital Stable Fund', 'Allianz Flexi Balanced Fund','Allianz MPF Age 65 Plus Fund','Allianz MPF Core Accumulation Fund', 'Allianz Stable Growth Fund']
Allianz_GroupA_list = ['Allianz Asian Fund', 'Allianz Greater China Fund', 'Allianz Oriental Pacific Fund', 'Allianz RMB Money Market Fund','Allianz Capital Stable Fund', 'Allianz Flexi Balanced Fund','Allianz MPF Age 65 Plus Fund','Allianz MPF Core Accumulation Fund', 'Allianz Stable Growth Fund']

Investco_chinese_fund_list = ['Investco Capital Stable Fund','Investco RMB Bond Fund','Investco Asian Equity Fund']
Investco_us_fund_list = ['Investco Age 65 Plus Fund','Investco Core Accumulation Fund','Investco Global Bond Fund']
Investco_GroupA_list =['Investco Capital Stable Fund','Investco RMB Bond Fund','Investco Asian Equity Fund','Investco Age 65 Plus Fund','Investco Core Accumulation Fund','Investco Global Bond Fund' ]

Fidelity_chinese_fund_list =['Fidelity RMB Bond Fund','Fidelity Asia Pacific Equity Fund']
Fidelity_us_fund_list =['Fidelity World Bond Fund','Fidelity Capital Stable Fund','Fidelity Age 65 Plus Fund', 'Fidelity Core Accumulation Fund','Fidelity Global Equity Fund']
Fidelity_GroupA_list = ['Fidelity RMB Bond Fund','Fidelity Asia Pacific Equity Fund','Fidelity World Bond Fund','Fidelity Capital Stable Fund','Fidelity Age 65 Plus Fund', 'Fidelity Core Accumulation Fund','Fidelity Global Equity Fund' ]

Allianz_GroupB_list = list(set(Allianz_fund_list) - set(Allianz_chinese_fund_list) - set(Allianz_us_fund_list))
Investco_GroupB_list = list(set(Investco_fund_list) - set(Investco_chinese_fund_list) - set(Investco_us_fund_list))
Fidelity_GroupB_list = list(set(Fidelity_fund_list) - set(Fidelity_chinese_fund_list) - set(Fidelity_us_fund_list))
print(Fidelity_GroupB_list)

['Fidelity MPF Conservative Fund', 'Fidelity Hong Kong Equity Fund', 'Fidelity Growth Fund', 'Fidelity Balanced Fund', 'Fidelity Hong Kong Bond Fund', 'Fidelity Hong Kong Tracker Fund']


## Load variables based on company chosen

In [30]:
if(company=='Investco'):
    fund_list = Investco_fund_list 
    GroupA_list = Investco_GroupA_list 
    GroupB_list = Investco_GroupB_list
elif(company=='Allianz'):
    fund_list = Allianz_fund_list 
    GroupA_list = Allianz_GroupA_list 
    GroupB_list = Allianz_GroupB_list
else:
    fund_list = Fidelity_fund_list 
    GroupA_list = Fidelity_GroupA_list 
    GroupB_list = Fidelity_GroupB_list



# Functions

In [31]:
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

In [32]:
def get_start_date():
    if (company=='Allianz'):
        start_date = date(2001,2,1)
    else:
        start_date = date(2000,12,1)
    return start_date

In [33]:
def random_number ():
    a_min = 0.35
    a_max = 1
    b_min = 0
    b_max = 0.65
    
    
    rand_fac1 = np.random.random(1)
    rand_fac2 = np.random.random(1)
    weight_a = a_min +(rand_fac1*(a_max-a_min))
    weight_b = b_min +(rand_fac2*(b_max-b_min))
    w_sum=weight_a + weight_b
    
    weight_a = weight_a/w_sum
    weight_b = weight_b/w_sum
    
    weight_a = float(weight_a)
    weight_b = float(weight_b)
    return weight_a, weight_b

In [34]:
def calculate_port_vol(df_daily,  month_index,A_name, B_name, A_weight, B_weight):
    #df_daily, df_merge_monthly, month_index, A_name, Available_B[list_num][1], A_weight, B_weight
    port_list =[]
    weights = np.array([A_weight, B_weight])

    port_list.append(A_name)
    port_list.append(B_name)
         
    #end_index= month_index.strftime('%Y-%m-%d')
    
    df_vol = df_daily.loc[:month_index]
    df_vol = select_columns(df_vol,port_list)
    
    df_vol.replace('', np.nan, inplace=True)
    df_vol = df_vol.dropna(how='any', axis=0)
    
    if(len(df_vol)==1):
        return ''
    
    cov_matrix_annual = df_vol.cov() * 252

    port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
    port_volatility = np.sqrt(port_variance)

    return port_volatility 

In [35]:
def generate_best_portfolio(df_daily, df_merge_monthly, df_next_month, month_index, next_month_index, A_name, A_return, Available_B, df_Available_B, portfolios_number):
    A_weight, B_weight = random_number()
    results=[]
 
    list_num = 0
    j=1
    port_return_list = np.zeros(portfolios_number*len(Available_B)) 
    
    for index, row in df_Available_B.iterrows():  #index is date, row is name #for each B
        if(row[j]==''):
            break
            
        for i in range(portfolios_number): #run _____ times for each Group B fund
            A_weight, B_weight = random_number()
            port_return = A_weight * A_return + B_weight * row[j]  #calculate the return
            port_return_list [i+list_num*portfolios_number] = port_return
            this_month_index=month_index.strftime('%Y-%m-%d')
            port_vol = calculate_port_vol (df_daily,  this_month_index, A_name, Available_B[list_num][1], A_weight, B_weight)           
            results.append([month_index, A_name, Available_B[list_num][1],A_weight, B_weight, port_return, port_vol,"","",""])
        list_num+=1
    
    df_results = pd.DataFrame(results,columns = ["Date","A_name","B_name","A_weight","B_weight","port_return","port_vol","flag","next_month_port_return","next_month_port_vol"])   
    df_results.set_index("Date", inplace = True) 
    return_max_index = port_return_list.argmax()
    df_best_B_port = pd.DataFrame(df_results.iloc[[return_max_index]])
    
    best_A = df_best_B_port.iloc[0]['A_name']
    best_B = df_best_B_port.iloc[0]['B_name']
    next_month_port_return = df_best_B_port.iloc[0]['A_weight']*df_next_month.iloc[-1][best_A ] + df_best_B_port.iloc[0]['B_weight']*df_next_month.iloc[-1][best_B]
    
    next_month_port_vol = calculate_port_vol(df_daily, next_month_index, best_A, best_B, df_best_B_port.iloc[0]['A_weight'], df_best_B_port.iloc[0]['B_weight'])
    
    df_best_B_port.at[month_index,'next_month_port_return']= next_month_port_return
    df_best_B_port.at[month_index,'next_month_port_vol']= next_month_port_vol

    
    #mark flag
    row_number = df_merge_monthly.index.get_loc(this_month_index)
    HSI_return = df_merge_monthly.iloc[row_number+1]['HSI Adj Close']
    HSI_vol = df_merge_monthly.iloc[row_number+1]['HSI_volatility']

    #if next_month_port_return >=df_best_B_port.iloc[0]['port_return'] and next_month_port_return >=HSI_return and next_month_port_vol <=HSI_vol:
    if next_month_port_return >=HSI_return and next_month_port_vol <=HSI_vol:
        df_best_B_port.at[month_index,'flag'] = 1
    else:
        df_best_B_port.at[month_index,'flag'] = 0
    
    
    return df_best_B_port

## HSI Monthly Return

In [36]:
def get_HSI_monthly():
    #read file and set index
    hsipath ="C:/Users/thy10/Desktop/AREIX/HSI.csv"  #read file
    df_HSI = pd.DataFrame(pd.read_csv(hsipath, index_col='Date', parse_dates=True),columns=['Adj Close']) #set df
    df_HSI.rename({"Adj Close":"HSI Adj Close"},axis = "columns", inplace=True)
    df_HSI.index.names = ['Valuation Date'] 
    df_HSI['HSI Adj Close'].replace('', np.nan, inplace=True)
    df_HSI=df_HSI.dropna(how='any', axis=0)

    #***for monthly return***
    df1=df_HSI
    df1.index = pd.to_datetime(df1.index, format="%y%m%d").to_period('M')
    
    #to get the first and last
    df_start=df1.groupby(df1.index).head(1)
    df_end=df1.groupby(df1.index).tail(1)
    
    df_HSI_monthly = df_start
    
    #calculate return
    for i in range(len(df_start)):
        if(df_end.iloc[i][0]==df_start.iloc[i][0]and i!=0):
            monthly_return = (df_end.iloc[i][0]-df_end.iloc[i-1][0])/df_end.iloc[i-1][0]
        else:
            monthly_return = (df_end.iloc[i][0]-df_start.iloc[i][0])/df_start.iloc[i][0]    
        df_HSI_monthly.iloc[i]['HSI Adj Close'] = monthly_return     
    
    #***for volatility***
    df_HSI_return = df_HSI.pct_change()
    HSI_vol_list = []
    for index, row in df_HSI_monthly.iterrows():
        this_month_index=index.strftime('%Y-%m-%d')
        HSI_vol_list.append(calculate_port_vol(df_HSI_return,  this_month_index, 'HSI Adj Close', 'HSI Adj Close', 0.5, 0.5))
    df_HSI_monthly['HSI_volatility'] = HSI_vol_list
    
    #output as csv
    #df_HSI_monthly.to_csv('df_HSI2_monthly.csv')
    return df_HSI_monthly

## Set company as df

In [37]:
df = pd.DataFrame(pd.read_csv("C:/Users/thy10/Desktop/AREIX/AREIX Jupyter/" + company+".csv", index_col='Valuation Date', parse_dates=True))
df=select_columns(df,fund_list) 
print("=== "+ company +" Saved As df ===")

#read monthly return csv
df_monthly_path = "C:/Users/thy10/Desktop/AREIX/AREIX Jupyter/"+company+"_df_monthly.csv"
df_monthly = pd.DataFrame(pd.read_csv(df_monthly_path, index_col='Valuation Date', parse_dates=True))
df_monthly.index = pd.to_datetime(df_monthly.index, format="%d%m%y").to_period('M')
df_monthly = df_monthly.fillna('')
if(company=='Fidelity'):
    df_monthly=df_monthly.drop(df_monthly.index[0])
print("=== "+ company +" Monthly Return Saved As df_monthy ===")

=== Fidelity Saved As df ===
=== Fidelity Monthly Return Saved As df_monthy ===


# Main function

In [38]:
#get HSI monthly return from funciton
df_HSI_monthly = get_HSI_monthly()

#df daily return
df_daily=df.pct_change()

#merge df_monthly with HSI monthly return
df_merge_monthly = pd.merge(df_monthly, df_HSI_monthly, on='Valuation Date', how='left')  #merge their return
#df_merge_monthly.to_csv('df_merge_monthly.csv')

<ipython-input-36-2536c9d9ce17>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HSI_monthly['HSI_volatility'] = HSI_vol_list


In [39]:
df_portfolios=[]
i=1

start_date=get_start_date()
end_date = start_date + relativedelta(months=+1)
start_index = start_date.strftime('%Y-%m')
end_index= end_date.strftime('%Y-%m')
print("Start Creating Porfolios...")
for index, row in df_merge_monthly.iterrows():  #for each month
    if(i==len(df_merge_monthly)-1):
        break

    df_next_month = df_merge_monthly.loc[start_index:end_index] #get next month
    
    Available_B = []
    for b in GroupB_list:  #create a df of B that is available at each month
        if row[b]!='':
            Available_B.append([index,b,row[b]])
    df_Available_B = pd.DataFrame(Available_B,columns = ["Valuation Date","B_name","B_return"])   #df of B that is available at each day
    df_Available_B.set_index("Valuation Date", inplace = True)
    
    for a in GroupA_list:  #for each A 
        #print("index is " + str(index) + "checking 1: A_name is " + a)
        if row[a]!='':
            df_portfolios.append(generate_best_portfolio (df_daily, df_merge_monthly, df_next_month,index, end_index,a, row[a], Available_B, df_Available_B, portfolios_number))
                                                    #df_merge_monthly, df_next_month, month_index, next_month_index, A_name, A_return, Available_B, df_Available_B, portfolios_number
        #print("=== Finished "+a+" ===")
    i+=1
    start_date = end_date
    end_date = start_date + relativedelta(months=+1)
    start_index = start_date.strftime('%Y-%m')
    end_index= end_date.strftime('%Y-%m')
    print("=== Finished "+str(index)+" ===")
print("Completed Creating Porfolios")    



Start Creating Porfolios...
=== Finished 2000-12 ===
=== Finished 2001-01 ===
=== Finished 2001-02 ===
=== Finished 2001-03 ===
=== Finished 2001-04 ===
=== Finished 2001-05 ===
=== Finished 2001-06 ===
=== Finished 2001-07 ===
=== Finished 2001-08 ===
=== Finished 2001-09 ===
=== Finished 2001-10 ===
=== Finished 2001-11 ===
=== Finished 2001-12 ===
=== Finished 2002-01 ===
=== Finished 2002-02 ===
=== Finished 2002-03 ===
=== Finished 2002-04 ===
=== Finished 2002-05 ===
=== Finished 2002-06 ===
=== Finished 2002-07 ===
=== Finished 2002-08 ===
=== Finished 2002-09 ===
=== Finished 2002-10 ===
=== Finished 2002-11 ===
=== Finished 2002-12 ===
=== Finished 2003-01 ===
=== Finished 2003-02 ===
=== Finished 2003-03 ===
=== Finished 2003-04 ===
=== Finished 2003-05 ===
=== Finished 2003-06 ===
=== Finished 2003-07 ===
=== Finished 2003-08 ===
=== Finished 2003-09 ===
=== Finished 2003-10 ===
=== Finished 2003-11 ===
=== Finished 2003-12 ===
=== Finished 2004-01 ===
=== Finished 2004-02 =

C:\Users\thy10\anaconda3\lib\site-packages\numpy\lib\function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\thy10\anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
C:\Users\thy10\anaconda3\lib\site-packages\pandas\core\frame.py:7605: RuntimeWarning: Degrees of freedom <= 0 for slice
  baseCov = np.cov(mat.T)
C:\Users\thy10\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\thy10\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


=== Finished 2013-06 ===
=== Finished 2013-07 ===
=== Finished 2013-08 ===
=== Finished 2013-09 ===
=== Finished 2013-10 ===
=== Finished 2013-11 ===
=== Finished 2013-12 ===
=== Finished 2014-01 ===
=== Finished 2014-02 ===
=== Finished 2014-03 ===
=== Finished 2014-04 ===
=== Finished 2014-05 ===
=== Finished 2014-06 ===
=== Finished 2014-07 ===
=== Finished 2014-08 ===
=== Finished 2014-09 ===
=== Finished 2014-10 ===
=== Finished 2014-11 ===
=== Finished 2014-12 ===
=== Finished 2015-01 ===
=== Finished 2015-02 ===
=== Finished 2015-03 ===
=== Finished 2015-04 ===
=== Finished 2015-05 ===
=== Finished 2015-06 ===
=== Finished 2015-07 ===
=== Finished 2015-08 ===
=== Finished 2015-09 ===
=== Finished 2015-10 ===
=== Finished 2015-11 ===
=== Finished 2015-12 ===
=== Finished 2016-01 ===
=== Finished 2016-02 ===
=== Finished 2016-03 ===
=== Finished 2016-04 ===
=== Finished 2016-05 ===
=== Finished 2016-06 ===
=== Finished 2016-07 ===
=== Finished 2016-08 ===
=== Finished 2016-09 ===


## Output as csv

In [40]:
df_portfolios_name = "df_portfolios_"+str(company)+".csv"
df_portfolios = pd.concat(df_portfolios)
df_portfolios.to_csv(df_portfolios_name)


## Details of the portfolios

In [41]:
print("--- Number of flags ---")
print(df_portfolios['flag'].value_counts())

--- Number of flags ---
1    506
0    453
Name: flag, dtype: int64
